In [42]:
import numpy as np
import math
from copy import deepcopy
from sklearn.cross_validation import KFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,recall_score,f1_score
import matplotlib.pyplot as plt
%matplotlib inline
import io

In [4]:
def create_data(data):
    z = data
    for j in z:
        j.insert(0,1)   
    return z

In [5]:
def find_alpha(class_labels):
    alpha1 = {}
    class_count1 = {}
    for i in class_labels:
        if i[0] not in class_count1.keys():
            class_count1[i[0]] = 1
        else:
            class_count1[i[0]] += 1
    classes1 = class_count1.keys()
    for j in  class_count1:
        alpha1[j] = class_count1[j]*1.0/len(class_labels)
    return classes1,class_count1,alpha1

In [6]:
def segregate_data(data,labels,clabels):
    multi_X = {}
    for i in range(len(clabels)):
        if clabels[i] not in multi_X.keys():
            multi_X[clabels[i]] = []
        for j in range(len(labels)):
            if labels[j][0] == clabels[i]:
                multi_X[clabels[i]].append(data[j]) 
    for i in multi_X:
        multi_X[i] = np.array(multi_X[i])
    return multi_X

In [86]:
def assume_weight(h,x):
    w = {}
    th = np.random.rand(h,len(x[0]))/10000
    #th = ((th-np.sum(th,axis =0))/np.sum(th,axis =0))/10000
    """th = th/(20000*np.sum(th))
    print th    """
    for i in range(h):
        w[i] = th[i]
    return w      

In [87]:
def assume_V(cl,h):
    v= {}
    th = np.random.rand(len(cl),h)/10000
    #th = ((th-np.sum(th,axis =0))/np.sum(th,axis =0))/100000000
    #th = th/(20000*np.sum(th))
    for i in cl:
        v[i] = th[i-1]
    return v      

In [9]:
def indicator(y,cl):
    ind = {}
    for k in cl:
        for l in y:
            if l[0] == k:
                if k not in ind.keys():
                    ind[k] = [[1]]
                else:
                    ind[k].append([1])
            else:
                if k not in ind.keys():
                    ind[k] = [[0]]
                else:
                    ind[k].append([0])
    return ind

In [10]:
def find_sigmoid1(w,data):
    sigmoid = []
    for i in data:
        tmp = []
        for j in w:
            x = np.dot(w[j].transpose(),i)
            tmp.append(1.0/(1+np.exp(-x)))
        sigmoid.append(tmp)
    sigmoig =create_data(sigmoid)
    return sigmoid

In [11]:
def find_sigmoid(w,data):
    sigmoid = {}
    for i in w:
        tmp = []
        for j in data:
            x = np.dot(w[i].transpose(),j)
            tmp.append([1.0/(1+np.exp(-x))])
        sigmoid[i] = np.array(tmp)
    return sigmoid

In [12]:
def softmax(theta,data):
    soft = {}
    for i in data:
        s = 0
        for j in theta:
            s = s + np.exp(np.dot(theta[j].transpose(),i))
        for k in theta:
            x = np.exp(np.dot(theta[k].transpose(),i))
            if k not in soft.keys():
                soft[k] = [[(x*1.0)/s]]
            else:
                soft[k].append([(x*1.0)/s])
    for i in soft:
        soft[i] = np.array(soft[i])
    return soft

In [62]:
def intrim(yh,y,v,j,k):
    s = 0
    for i in v:
        x = yh[i][k]- y[i][k]
        s = s + x*v[i][j]
    return s

In [68]:
def grad_desc(W,V,data,indic,lr,ic):
    for cnt in range(ic):
        zy = find_sigmoid1(W,data)
        zz = find_sigmoid(W,data)
        yhat = softmax(V,zy)
        for m in yhat:
            x = yhat[m] - indic[m]
            xz = x*zy
            V[m] = V[m] - (lr*np.sum(xz,axis=0))
        for j in W:
            s = 0
            for i in range(len(data)):
                s = s + intrim(yhat,indic,V,j,i)*zy[i][j]*(1-zy[i][j])*data[i]
            W[j] = W[j] - (lr*s)
    return V,W

In [61]:
def grad_desc_m(W,V,data,indic,lr,ic):
    prevw = {}
    for cnt in range(ic):
        zy = find_sigmoid1(W,data)
        zz = find_sigmoid(W,data)
        yhat = softmax(V,zy)
        for m in yhat:
            x = yhat[m] - indic[m]
            xz = x*zy
            V[m] = V[m] - (lr*np.sum(xz,axis=0))
        prevw[cnt] = W
        for j in W:
            s = 0
            for i in range(X.shape[0]):
                print i
                s = s + intrim(yhat,indic,V,j,i)*zy[j][i]*(1-zy[j][i])*data[i]
            if cnt != 0:
                W[j] = W[j] - (lr*s) + (0.2*prevw[cnt-1][j])
    return V,W

In [16]:
def prediction(w,v,data):
    Zp = find_sigmoid1(w,data)
    pre = softmax(v,Zp)  
    ll = []
    for j in range(len(pre.values()[0])):
        tmp= []
        for i in pre:
            tmp.append((i,pre[i][j][0]))
        ll.append(tmp)
    predi =[]
    for i in ll:
            predi.append([max(i,key=lambda item:item[1])[0]])
    return np.array(predi)

In [17]:
def mean_squrae_error(pred,y):
    return sum([(i-j)**2 for i,j in zip(pred,y)])/float(len(pred))

In [18]:
def find_confusion_matrix(clabels,actual,predicted):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        for j in range(len(actual)):
            if actual[j][0] == i and actual[j][0] == predicted[j][0]:
                tmp[clabels.index(i)] += 1
            elif actual[j][0] == i and actual[j][0] != predicted[j][0]:
                tmp[clabels.index(predicted[j][0])] += 1
        cm.append(tmp)
    return np.array(cm)
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec
def find_fmeasure(prec,rec):
    tmp = []
    for i,j in zip(prec,rec):
        tmp.append(2.0*(i*j)/(i+j))
    return tmp

In [19]:
def roc(clabels,acutal,predicted):
    confmatrix = find_confusion_matrix(clabels,acutal,predicted)
    precision = find_precision(confmatrix)
    recall = find_recall(confmatrix)
    return precision,recall,confmatrix

In [112]:
def cross_validation(data, labels,clabels, n_folds=10):
    cv = KFold(len(labels), n_folds,shuffle= True)
    precision_list = []
    confusion = []
    recall_list = []
    training_MSE_list =[]
    testing_MSE_list = []
    i = 0
    for train_ind, test_ind in cv: 
        train_V,train_W = training(data[train_ind], labels[train_ind],4,clabels,0.000454,1000)
        training_MSE = mean_squrae_error(prediction(train_W,train_V,data[train_ind]), labels[train_ind])
        training_MSE_list.append(training_MSE)
        predict = prediction(train_W,train_V,data[test_ind])
        #p,r,cm = roc(clabels,labels[test_ind],predict)
        #precision_list.append(p)
        #recall_list.append(r)
        confusion.append(find_confusion_matrix(clabels,labels[test_ind],predict))
        testing_MSE_list.append(mean_squrae_error(predict,labels[test_ind]))
    for i in range(len(testing_MSE_list)):
        print 'Fold',i,'Testing Error',testing_MSE_list[i]
    print "Average Mean Square Error"
    print "Training Error \t Testing Error"
    print np.mean(training_MSE_list),"\t",np.mean(testing_MSE_list)
    return training_MSE_list,confusion

In [21]:
def evaluation(clabels,acutal,predicted):
    confmatrix = find_confusion_matrix(clabels,acutal,predicted)
    print "Confusion Matrix"
    print confmatrix
    accuracy = find_accuracy(confmatrix)
    print "Accuracy", accuracy
    precision = find_precision(confmatrix)
    print "Precision", precision
    recall = find_recall(confmatrix)
    print "Recall", recall
    f_score =find_fmeasure(precision,recall)
    print "F_score", f_score

In [22]:
filename = "iris.data.txt"
r = io.open(filename, encoding='utf8').readlines()
X = []
Y = []
for i in r[0:150]:
    x = i.split(',')
    X.append(map(float,x[0:len(x)-1]))
    if x[-1] == "Iris-setosa\n":
        Y.append([1])
    elif x[-1] == 'Iris-versicolor\n':
        Y.append([2])
    elif x[-1] == 'Iris-virginica\n':
        Y.append([3])
ZX = deepcopy(X)
ZX = np.array(create_data(ZX))
X = np.array(X)
Y = np.array(Y)

In [23]:
classes,class_count,alpha = find_alpha(Y)

In [26]:
def training(data,y,h,cl,lr,ic):
    W0 = assume_weight(h,ZX)
    V0 = assume_V(classes,(h+1))
    ind = indicator(y,cl)
    Vf,Wf = grad_desc(W0,V0,data,ind,lr,ic)
    return Vf,Wf

In [39]:
V,W = training(ZX,Y,4,classes,0.00000000454,400)

In [40]:
P = prediction(W,V,ZX)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(ZX, Y, test_size=0.4, random_state=42)

In [45]:
classes,class_count,alpha = find_alpha(y_train)

In [92]:
V,W = training(X_train,y_train,4,classes,0.00454,1000)

In [93]:
P = prediction(W,V,X_test)

In [94]:
evaluation(classes,y_test,P)

Confusion Matrix
[[23  0  0]
 [ 0 18  1]
 [ 0  0 18]]
Accuracy 0.983333333333
Precision [1.0, 1.0, 0.94736842105263153]
Recall [1.0, 0.94736842105263153, 1.0]
F_score [1.0, 0.97297297297297303, 0.97297297297297303]


In [113]:
mse,cmatrix = cross_validation(ZX,Y,classes)

Fold 0 Testing Error [ 0.4]
Fold 1 Testing Error [ 0.4]
Fold 2 Testing Error [ 0.2]
Fold 3 Testing Error [ 0.46666667]
Fold 4 Testing Error [ 0.]
Fold 5 Testing Error [ 0.33333333]
Fold 6 Testing Error [ 0.53333333]
Fold 7 Testing Error [ 0.4]
Fold 8 Testing Error [ 0.33333333]
Fold 9 Testing Error [ 0.4]
Average Mean Square Error
Training Error 	 Testing Error
0.317037037037 	0.346666666667


In [209]:
np.mean(cmatrix,axis=0)

array([[ 5. ,  0. ,  0. ],
       [ 0.2,  1.2,  3.6],
       [ 0. ,  1.4,  3.6]])

In [ ]:
V1,W1 = training(X_train,y_train,5,classes,0.00454,1000)

In [ ]:
P1 = prediction(W1,V1,X_test)

In [227]:
evaluation(classes,y_test,P1)

Confusion Matrix
[[23  0  0]
 [ 0  5 14]
 [ 0  0 18]]
Accuracy 0.766666666667
Precision [1.0, 1.0, 0.5625]
Recall [1.0, 0.26315789473684209, 1.0]
F_score [1.0, 0.41666666666666669, 0.71999999999999997]


In [185]:
V2,W2 = training(X_train,y_train,2,classes,0.000000000004954,110)

In [186]:
P2 = prediction(W2,V2,X_test)

In [187]:
evaluation(classes,y_test,P2)

Confusion Matrix
[[ 0  0 23]
 [ 0  0 19]
 [ 0  0 18]]
Accuracy 0.3
Precision [nan, nan, 0.29999999999999999]
Recall [0.0, 0.0, 1.0]
F_score [nan, nan, 0.46153846153846151]


c:\python27\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [189]:
V3,W3 = training(X_train,y_train,8,classes,0.00454,500)
P3 = prediction(W3,V3,X_test)
evaluation(classes,y_test,P3)

Confusion Matrix
[[23  0  0]
 [ 0 12  7]
 [ 0  0 18]]
Accuracy 0.883333333333
Precision [1.0, 1.0, 0.71999999999999997]
Recall [1.0, 0.63157894736842102, 1.0]
F_score [1.0, 0.77419354838709675, 0.83720930232558133]
